In [10]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
import csv
pd.set_option('display.max_columns', None)  
warnings.filterwarnings('ignore')

In [11]:
navco = pd.read_stata('NAVCO2-1.dta')
navco = navco.fillna(0)

sanctions = pd.read_csv('sanctions.csv', sep = ';', encoding = 'cp1251')
sanctions = sanctions.fillna(0)

gdp = pd.read_csv('GDP.csv', sep = ',', encoding = 'cp1251')
gdp = gdp.fillna(0)

vdem = pd.read_csv('vdem.csv', sep = ',', encoding = 'cp1251')
vdem = vdem.fillna(0)

reign = pd.read_csv('leader_list.csv', sep = ',',  encoding = 'cp1251', on_bad_lines='skip')
reign = reign.fillna(0)
reign['eyear'] = reign['eyear'].astype('int64')
reign['syear'] = reign['syear'].astype('int64')

gwf_autocratic = pd.read_csv('gwf_autocratic.csv', encoding = 'cp1251')
gwf_autocratic.rename(columns={'gwf_cowcode': 'cow'}, inplace = True)
gwf_autocratic['year'] = gwf_autocratic['year'].astype('int64')
gwf_autocratic['cow'] = gwf_autocratic['cow'].astype('int64')
gwf_autocratic = gwf_autocratic[['year', 'cow', 'gwf_regimetype', 'gwf_duration']]
gwf_autocratic = gwf_autocratic.fillna(0)

navco['protestnum'] = 1
navco = navco[['year', 'loc_cow', 'protestnum']]
protest = navco.groupby(['loc_cow', 'year']).agg(sum)
protest = protest.reset_index()
protest.rename(columns={'loc_cow': 'cow'}, inplace=True)

sanctions['sanctionnum'] = 1
sanctions['startyear'] = sanctions['startyear'].astype('int64')
sanctions['targetstate'] = sanctions['targetstate'].astype('int64')
sanctions = sanctions[['startyear', 'targetstate', 'sanctionnum']]
sanctions = sanctions.groupby(['targetstate', 'startyear']).agg(sum)
sanctions = sanctions.reset_index()
sanctions.rename(columns={'startyear': 'year', 'targetstate': 'cow'}, inplace=True)
sanctions['year'] = sanctions['year'].astype('int64')

In [12]:
navco.rename(columns={'loc_cow': 'cow', 'id': 'protest_id'}, inplace=True)
gdp.rename(columns={'date': 'year', 'NY.GDP.PCAP.CD': 'gdpval'}, inplace = True)
vdem.rename(columns={'vdem_cowcode': 'cow'}, inplace = True)

In [13]:
protest['cow'] = protest['cow'].astype('int64')
protest['year'] = protest['year'].astype('int64')

sanctions['year'] = sanctions['year'].astype('int64')
sanctions['cow'] = sanctions['cow'].astype('int64')

gdp['year'] = gdp['year'].astype('int64')
gdp['cow'] = gdp['cow'].astype('int64')
gdp = gdp[['iso3c', 'country', 'year', 'gdpval', 'cow', 'region']]

vdem['year'] = vdem['year'].astype('int64')
vdem['cow'] = vdem['cow'].astype('int64')
vdem = vdem[['year', 'cow', 'v2x_polyarchy']]

In [5]:
d = []
for a, row in reign.iterrows():
    for i in range(0, row['eyear'] - row['syear']):
#         print(row['stateabb'], row['ccode'], row['syear'] + i, " ", i)
        d.append(
         {
#             'stateabb': row['stateabb'],
            'ccode': row['ccode'],
            'year':  row['syear'] + i,
            'tenure': i,
            'tenure_lagged': i+1,
            'leader_name': row['leader']
        }
    )
# d = pd.DataFrame()
tenure_data = pd.DataFrame(d)
# tenure_data.to_csv('tenure_data.csv')

tenure_data.rename(columns={'ccode': 'cow'}, inplace = True)
tenure_data['year'] = tenure_data['year'].astype('int64')
tenure_data['cow'] = tenure_data['cow'].astype('int64')
tenure_data = tenure_data.fillna(0)

In [14]:
df1 = pd.merge(gdp, protest, on = ['cow', 'year'], how = 'left')
df2 = pd.merge(df1, gwf_autocratic, on = ['cow', 'year'], how = 'left')
df3 = pd.merge(df2, vdem, on = ['cow', 'year'], how = 'left')
df4 = pd.merge(df3, tenure_data, on = ['cow', 'year'], how = 'left')
df5 = pd.merge(df4, sanctions, on = ['cow', 'year'], how = 'left')
finaldf = df5[(df5['year'] >= 1950) & (df5['year'] <= 2005)]
finaldf = finaldf.fillna(0)
finaldf = finaldf.drop_duplicates(keep = 'first')
# finaldf.to_csv('essay.csv')

In [15]:
finaldf

,iso3c,country,year,gdpval,cow,region,protestnum,gwf_regimetype,gwf_duration,v2x_polyarchy,tenure,tenure_lagged,leader_name,sanctionnum
0,AFG,Afghanistan,1960,62.369375,700,South Asia,0.0,monarchy,31.0,0.080,7.0,8.0,Sardar Mohammad Daud Khan,0.0
1,AFG,Afghanistan,1961,62.443703,700,South Asia,0.0,monarchy,32.0,0.083,8.0,9.0,Sardar Mohammad Daud Khan,1.0
2,AFG,Afghanistan,1962,60.950364,700,South Asia,0.0,monarchy,33.0,0.082,9.0,10.0,Sardar Mohammad Daud Khan,0.0
3,AFG,Afghanistan,1963,82.021738,700,South Asia,0.0,monarchy,34.0,0.085,0.0,1.0,Mohammad Yusuf,0.0
4,AFG,Afghanistan,1964,85.511073,700,South Asia,0.0,monarchy,35.0,0.137,1.0,2.0,Mohammad Yusuf,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12074,ZWE,Zimbabwe,2001,569.003209,552,Sub-Saharan Africa,0.0,party-based,21.0,0.269,21.0,22.0,Mugabe,1.0
12075,ZWE,Zimbabwe,2002,529.186883,552,Sub-Saharan Africa,0.0,party-based,22.0,0.268,22.0,23.0,Mugabe,0.0
12076,ZWE,Zimbabwe,2003,474.302201,552,Sub-Saharan Africa,0.0,party-based,23.0,0.268,23.0,24.0,Mugabe,0.0
12077,ZWE,Zimbabwe,2004,477.399491,552,Sub-Saharan Africa,0.0,party-based,24.0,0.255,24.0,25.0,Mugabe,0.0
